Credit : https://github.com/nicolashernandez/

In [8]:
import pandas as pd
from nltk.stem.snowball import FrenchStemmer
import re
import nltk
from nltk.tokenize import word_tokenize

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahamid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
df = pd.read_csv('C:/Users/ahamid/Downloads/archive/Comments.csv', encoding='utf-8')
df = df[['Name','Comment']]
df = df.sample(100)
df.reset_index(drop=True, inplace=True)
df.head()

,Name,Comment
0,Maaf,La MAAF est une assurance qui me parait cher a...
1,Eca Assurances,Une tromperie de haut niveau
2,Alptis,"J'ai été adhérent pendant des années, très con..."
3,Néoliane Santé,Très bonne mutuelle 48 heures les retransmissi...
4,Mgen,"Depuis longtemps à la MGEN, dégradation des va..."


### Clean words from special chars

In [17]:
def get_word_tokens(string, maxsplit=0):
    # Tokenize the URL
    delimiters = "-", "/", ".", ":", "_", "+", "=", "&", "*", ",", "?", "!", "$", "(", ")", "[", "]", "@", "^", "\\", "'", " "
    regex_pattern = '|'.join(map(re.escape, delimiters))
    tokens = re.split(regex_pattern, string, maxsplit)
    filtered_tokens = filter(None, tokens)
    list_of_tokens = list(filtered_tokens)
    tokens = ' '.join(list_of_tokens)
    return tokens

In [18]:
tokenized_subject = df.apply(lambda row: get_word_tokens(row['Comment']), axis=1)
df.insert(len(df.columns), 'tokenized_Comment', tokenized_subject)

In [19]:
pd.DataFrame(df.loc[3])

,3
Name,Néoliane Santé
Comment,Très bonne mutuelle 48 heures les retransmissi...
tokenized_Comment,Très bonne mutuelle 48 heures les retransmissi...


### nltk snowball stemmer

In [20]:
subs = list(df['tokenized_Comment'])

### exemple Besoin de text preprocessing :

In [21]:
print(subs[3])

Très bonne mutuelle 48 heures les retransmissions 
j y suis depuis 2 ans rien à signaler service gestion à l écoute à chaque demande merci à vous


In [22]:
print("subject d'origine :", df['Comment'][3])
print("\nsubject cleaned :", subs[3])
print("\nsubject stemmed :", [FrenchStemmer().stem(w) for w in word_tokenize(subs[3], language='french')])

subject d'origine : Très bonne mutuelle 48 heures les retransmissions...
j'y suis depuis 2 ans rien à signaler service gestion à l'écoute à chaque demande. merci à vous.

subject cleaned : Très bonne mutuelle 48 heures les retransmissions 
j y suis depuis 2 ans rien à signaler service gestion à l écoute à chaque demande merci à vous

subject stemmed : ['tres', 'bon', 'mutuel', '48', 'heur', 'le', 'retransm', 'j', 'y', 'suis', 'depuis', '2', 'an', 'rien', 'à', 'signal', 'servic', 'gestion', 'à', 'l', 'écout', 'à', 'chaqu', 'demand', 'merc', 'à', 'vous']


### spacy ????????????????????<br>
**Install spacy based on yout conf**
https://spacy.io/usage

In [23]:
# !python -m spacy download fr_core_news_sm 
# !python -m spacy download fr_dep_news_trf 
# !pip install spacy-transformers

In [24]:
import spacy
nlp = spacy.load('fr_core_news_sm')

In [25]:
print("subject d'origine :", df['Comment'][3])
print("\nsubject cleaned :", subs[3])
print("\nsubject stemmed :", [FrenchStemmer().stem(w) for w in word_tokenize(subs[3], language='french')])
print("\nsubject lemmatized :", [token.lemma_ for token in nlp(subs[3])])

subject d'origine : Très bonne mutuelle 48 heures les retransmissions...
j'y suis depuis 2 ans rien à signaler service gestion à l'écoute à chaque demande. merci à vous.

subject cleaned : Très bonne mutuelle 48 heures les retransmissions 
j y suis depuis 2 ans rien à signaler service gestion à l écoute à chaque demande merci à vous

subject stemmed : ['tres', 'bon', 'mutuel', '48', 'heur', 'le', 'retransm', 'j', 'y', 'suis', 'depuis', '2', 'an', 'rien', 'à', 'signal', 'servic', 'gestion', 'à', 'l', 'écout', 'à', 'chaqu', 'demand', 'merc', 'à', 'vous']

subject lemmatized : ['très', 'bon', 'mutuelle', '48', 'heure', 'le', 'retransmission', '\r\n', 'j', 'y', 'être', 'depuis', '2', 'an', 'rien', 'à', 'signaler', 'service', 'gestion', 'à', 'l', 'écoute', 'à', 'chaque', 'demande', 'merci', 'à', 'vous']


In [26]:
# spacy lemmatization
import spacy_transformers
nlp = spacy.load('fr_dep_news_trf') # accuracy : CELUI là marche mieux contextuellement et le premier est plus fonctionnel
print([token.lemma_ for token in nlp(subs[3])])

['très', 'bon', 'mutuelle', '48', 'heure', 'le', 'retransmission', '\r\n', 'j', 'y', 'suivre', 'depuis', '2', 'an', 'rien', 'à', 'signaler', 'service', 'gestion', 'à', 'l', 'écoute', 'à', 'chaque', 'demande', 'merci', 'à', 'vous']


### FrenchLefffLemmatizer


In [27]:
# !pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git

In [28]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

french_lefff_lemmatizer = FrenchLefffLemmatizer()

In [29]:
spacy_to_lefff_pos = {
    "ADJ": "adj",
    "ADP": "det",
    "ADV": "adv",
    "DET": "det",
    "PRON": "cln",
    "PROPN": "np",
    "NOUN": "nc",
    "VERB": "v",
    "PUNCT": "poncts"
}

In [30]:
def french_lefff_lemmatizer_context_sensitive_wi_spacy_pos (french_lefff_lemmatizer, spacy_doc):
    ''' 
    Given that lefff returns wrong lemma for DET and PRON
    Given that spacy returns wrong lemma for VERB
    Returns the spacy lemma except for VERB
    '''
    lemmas = []
    for t in spacy_doc:
        print ([t.text,t.pos_,t.lemma_, spacy_to_lefff_pos[t.pos_] if t.pos_ in spacy_to_lefff_pos else t.pos_, french_lefff_lemmatizer.lemmatize(t.text.lower()), french_lefff_lemmatizer.lemmatize(t.text,spacy_to_lefff_pos[t.pos_] if t.pos_ in spacy_to_lefff_pos else '')])
        if t.pos_ in ['VERB']:
            print ('to fix')
            lefff_lemma = french_lefff_lemmatizer.lemmatize(t.text, spacy_to_lefff_pos[t.pos_])
            if type(lefff_lemma) != type ("") and len(lefff_lemma) !=0:
                lefff_lemma = lefff_lemma[0][0]
                print ('is a list so lemma is : '+lefff_lemma)
            else: 
                lefff_lemma =lefff_lemma
                print ('is a string so lemma is : '+lefff_lemma)
            lemmas.append(lefff_lemma)
        else:
            lemmas.append(t.lemma_)

    return lemmas

In [31]:
text = subs[3]
text

'Très bonne mutuelle 48 heures les retransmissions \r\nj y suis depuis 2 ans rien à signaler service gestion à l écoute à chaque demande merci à vous'

In [32]:
print(french_lefff_lemmatizer_context_sensitive_wi_spacy_pos(french_lefff_lemmatizer, nlp(text)))

['Très', 'ADV', 'très', 'adv', 'très', [('très', 'adv')]]
['bonne', 'ADJ', 'bon', 'adj', 'bonne', [('bon', 'adj')]]
['mutuelle', 'NOUN', 'mutuelle', 'nc', 'mutuelle', [('mutuelle', 'nc')]]
['48', 'NUM', '48', 'NUM', '48', []]
['heures', 'NOUN', 'heure', 'nc', 'heure', [('heure', 'nc')]]
['les', 'DET', 'le', 'det', 'les', [('le', 'det')]]
['retransmissions', 'NOUN', 'retransmission', 'nc', 'retransmission', [('retransmission', 'nc')]]
['\r\n', 'SPACE', '\r\n', 'SPACE', '\r\n', []]
['j', 'PRON', 'j', 'cln', 'j', []]
['y', 'PRON', 'y', 'cln', 'y', []]
['suis', 'VERB', 'suivre', 'v', 'suis', 'être']
to fix
is a string so lemma is : être
['depuis', 'ADP', 'depuis', 'det', 'depuis', []]
['2', 'NUM', '2', 'NUM', '2', []]
['ans', 'NOUN', 'an', 'nc', 'an', [('an', 'nc')]]
['rien', 'PRON', 'rien', 'cln', 'rien', []]
['à', 'ADP', 'à', 'det', 'à', []]
['signaler', 'VERB', 'signaler', 'v', 'signaler', 'signaler']
to fix
is a string so lemma is : signaler
['service', 'NOUN', 'service', 'nc', 'servic

.

## spacy-lefff
**Custom French POS and lemmatizer based on Lefff for spacy**

In [33]:
# !pip install spacy-lefff

In [34]:
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger
from spacy.language import Language

@Language.factory('french_lemmatizer')
def create_french_lemmatizer(nlp, name):
    return LefffLemmatizer(after_melt=True, default=True)

@Language.factory('melt_tagger')  
def create_melt_tagger(nlp, name):
    return POSTagger()
 
nlp = spacy.load('fr_core_news_sm') # efficiency

nlp.add_pipe('melt_tagger', after='parser')
nlp.add_pipe('french_lemmatizer', after='melt_tagger')
doc = nlp(u"Apple cherche a acheter une startup anglaise pour 1 milliard de dollard")
for d in doc:
    print(d.text, d.pos_, d._.melt_tagger, d._.lefff_lemma, d.tag_, d.lemma_)

Apple NOUN NPP apple NOUN apple
cherche NOUN V chercher NOUN cherche
a AUX V avoir AUX avoir
acheter VERB VINF acheter VERB acheter
une DET DET un DET un
startup NOUN NC startup NOUN startup
anglaise ADJ ADJ anglais ADJ anglais
pour ADP P pour ADP pour
1 NUM DET 1 NUM 1
milliard NOUN NC milliard NOUN milliard
de ADP P de ADP de
dollard PROPN NC dollard PROPN dollard


# stanza aka Neural Stanford corenlp
Aside from the neural pipeline, this package also includes an official wrapper for accessing the Java Stanford CoreNLP software with Python code https://github.com/stanfordnlp/CoreNLP.

800 Mo of parameters + 600 for the French model (stored in ~/stanza_resources/fr/default.zip)

https://github.com/stanfordnlp/stanza

In [35]:
# !pip3 install stanza

In [36]:
import stanza
stanza.download('fr')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('fr') # This sets up a default neural pipeline in English
# doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc = nlp(text)

doc.sentences[0].print_dependencies()

for sentence in doc.sentences:
    for word in sentence.words:
        print(word.text, word.lemma, word.pos)

Correct lemmatization on VERBs ; incorrect lemmatization for PRONouns

# treetagger-python
A Python module for interfacing with the Treetagger by Helmut Schmid

Wrapper source https://github.com/miotto/treetagger-python (alternative exists)
License GPL-v3
TreeTagger source https://www.cis.lmu.de/~schmid/tools/TreeTagger/
A neural version exists too. It lemmatizes all tokens. Lemmas of unknown tokens are guessed and are therefore not guaranteed to be always correct. Slower, requires PyTorch, requires a GPU for improved speed, larger parameter files https://www.cis.lmu.de/~schmid/tools/RNNTagger/ (3.1 GB)


## CLTK, The Classical Language Toolkit
The Classical Language Toolkit (CLTK) is a Python library offering natural language processing (NLP) for pre-modern languages.

Home https://github.com/cltk/cltk
Licence MIT
Doc https://docs.cltk.org/en/latest/quickstart.html

In [37]:
# !pip install --user cltk

In [40]:
from cltk import NLP

#  Middle French language
cltk_nlp = NLP(language="mfr")
cltk_doc = cltk_nlp.analyze(text=text)
print(cltk_doc.lemmata)